In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as BS
import re
import requests
from urllib import request
import pandas as pd

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time

In [ ]:
chromedriver = 'D:/20220103_lab/14.python/data/chrome/chromedriver'
driver = webdriver.Chrome(chromedriver) 

# 포스팅 작성 당시 크롬 버젼 : 92

In [ ]:
df = pd.DataFrame

In [ ]:
# 네이버 지도 검색창에 [~동 @@식당]으로 검색해 정확도를 높여야 합니다. 검색어를 미리 설정해줍시다.

df['naver_keyword'] = df['dong'] + "%20" + df['name']  # "%20"는 띄어쓰기를 의미합니다.
df['naver_map_url'] = ''

In [ ]:
# 본격적으로 가게 상세페이지의 URL을 가져옵시다

for i, keyword in enumerate(df['naver_keyword'].tolist()):
    print("이번에 찾을 키워드 :", i, f"/ {df.shape[0] -1} 행", keyword)
    try:
        naver_map_search_url = f"https://m.map.naver.com/search2/search.naver?query={keyword}&sm=hty&style=v5"
        
        driver.get(naver_map_search_url)
        time.sleep(3.5)
        df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
        # 네이버 지도 시스템은 data-cid에 url 파라미터를 저장해두고 있었습니다.
        # data-cid 번호를 뽑아두었다가 기본 url 템플릿에 넣어 최종적인 url을 완성하면 됩니다.
        
        #만약 검색 결과가 없다면?
    except Exception as e1:
        if "li:nth-child(1)" in str(e1):  # -> "child(1)이 없던데요?"
            try:
                df.iloc[i,-1] = driver.find_element_by_css_selector("#ct > div.search_listview._content._ctList > ul > li:nth-child(1) > div.item_info > a.a_item.a_item_distance._linkSiteview").get_attribute('data-cid')
                time.sleep(1)
            except Exception as e2:
                print(e2)
                df.iloc[i,-1] = np.nan
                time.sleep(1)
        else:
            pass


driver.quit()

In [ ]:
# 이때 수집한 것은 완전한 URL이 아니라 URL에 들어갈 ID (data-cid 라는 코드명으로 저장된) 이므로, 온전한 URL로 만들어줍니다

df['naver_map_url'] = "https://m.place.naver.com/restaurant/" + df['naver_map_url']

In [ ]:
# URL이 수집되지 않은 데이터는 제거합니다.
df = df.loc[~df['naver_map_url'].isnull()]

In [ ]:
# 크롤링 에러가 떠서 'null'을 넣어 둔 데이터는 활용 의미가 없으므로 행 삭제를 해줘도 됩니다
df = df.loc[~(df['naver_store_type'].str.contains('null'))]


# 별점 평균, 수 같은 데이터 역시 스트링 타입으로 크롤링이 되었으므로 numeric으로 바꿔줍니다.
df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']] = df[['naver_star_point', 'naver_star_point_qty', 'naver_blog_review_qty']].apply(pd.to_numeric)
Python


In [ ]:
from selenium import webdriver
import time

# 크롬창(웹드라이버) 열기
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
# 구글 지도 접속하기
driver.get("https://www.google.com/maps/")

In [ ]:
# 검색창에 "카페" 입력하기
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

In [ ]:
# 검색버튼 누르기
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

In [ ]:
# 여러 페이지(999)에서 반복하기
for i in range(999):
    time.sleep(3)

    # 컨테이너(가게) 데이터 수집 // div.section-result-content
    stores = driver.find_elements_by_css_selector("div.section-result-content")

    for s in stores:
        # 가게 이름 데이터 수집 // h3.section-result-title
        title = s.find_element_by_css_selector("h3.section-result-title").text

        # 평점 데이터 수집 // span.cards-rating-score
        # 평점이 없는 경우 에러 처리
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"

            # 가게 주소 데이터 수집 // span.section-result-location

            addr = s.find_element_by_css_selector("span.section-result-location").text

            print(title, "/", score, "/", addr)
            
    # 다음페이지 버튼 클릭 하기
    # 다음페이지가 없는 경우(데이터 수집 완료) 에러 처리
    try:
        nextpage = driver.find_element_by_css_selector("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [ ]:
# 크롬창 닫기
driver.close()
# 출처: https://datalabbit.tistory.com/45 [간토끼 DataMining Lab:티스토리]

In [ ]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [ ]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

In [ ]:
driver.get("https://www.google.com/maps/")

In [ ]:
searchbox = driver.find_element_by_css_selector("input#searchboxinput")
searchbox.send_keys("동물병원")

In [ ]:
searchbutton = driver.find_element_by_css_selector("button#searchbox-searchbutton")
searchbutton.click()

In [ ]:
for i in range(999):
    time.sleep(3)
    stores = driver.find_elements_by_css_selector("m6QErb DxyBCb kA9KIf dS8AEf ecceSd")
    for s in stores:
        title = s.find_element_by_css_selector("h3.section-result-title").text
        try:
            score = s.find_element_by_css_selector("span.cards-rating-score").text
        except:
            score = "평점없음"
            
            addr = s.find_element_by_css_selector("span.section-result-location").text
            
            print(title, "/", score, "/", addr)
            
    try:
        nextpage = driver.find_element_by_css_selector("button#ppdPk-Ej1Yeb-LgbsSe-tJiF1e")
        nextpage.click()
    except:
        print("데이터 수집 완료.")
        break

In [ ]:
driver.close()

In [1]:
from selenium import webdriver
import time
from selenium.webdriver.common.by import By

In [2]:
driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")

C:\Users\82103\AppData\Local\Temp/ipykernel_1656/3310334687.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("D:/20220103_lab/14.python/data/chrome/chromedriver")


In [3]:
driver.get("https://www.google.com/maps/")

In [4]:
search_input_tag = driver.find_element(By.ID, 'searchboxinput')
search_input_tag.send_keys('동물병원')
time.sleep(1)

In [5]:
search_button_tag = driver.find_element(By.ID, 'searchbox-searchbutton')
search_button_tag.click()
time.sleep(5)

In [19]:
target = driver.find_element(By.XPATH, '/html/body/div[3]/div[9]/div[9]/div[1]/div[1]/div[1]/div[2]')

In [20]:
target.text

'평점\n영업시간\n모든 필터\n아마존동물병원\n4.6(25)\n동물병원 · 임학동 67-7\n영업 종료 ⋅ 오전 10:00에 영업 시작 · 032-555-7502 ext. 3\n경로\n계양동물메디컬센터\n4.3(50)\n동물병원 · 계산동 942-24번지\n영업 종료 ⋅ 오전 9:30에 영업 시작 · 032-555-9975\n경로\n한솔동물병원\n4.3(13)\n동물병원 · 계산2동 918-2\n032-553-4652\n웹사이트\n경로\n해림동물병원\n4.8(100)\n동물병원 · 작전동 864-39\n영업 종료 ⋅ 오전 10:00에 영업 시작 · 032-546-0079\n경로\n이지동물병원\n4.5(8)\n동물 병원 · 병방동 422\n영업 종료 ⋅ 오전 10:00에 영업 시작 · 032-549-0077\n경로\n신통방통동물병원\n4.9(10)\n동물 병원 · 작전동 863\n영업 종료 ⋅ 오전 10:00에 영업 시작 · 032-552-3369\n경로\n다오동물병원\n4.7(6)\n동물병원 · 계산동 1062\n032-548-0275\n경로\n재영동물병원\n3.8(4)\n동물병원 · 계산1동\n032-542-4647\n경로\n신영재동물병원\n4.0(2)\n동물병원 · 병방동 426\n032-552-0075\n웹사이트\n경로\n작전24동물병원\n4.4(7)\n동물병원 · 작전동 925-1\n경로\n아프리카 동물의료센터 [구 아프리카동물병원]\n4.4(89)\n동물병원 · 장제로340번길 5 3층\n영업 종료 ⋅ 오전 9:00에 영업 시작 · 032-508-7582\n웹사이트\n경로\n부평종합동물병원\n4.5(101)\n동물병원 · 부평동 부평대로 154\n영업 종료 ⋅ 오전 9:30에 영업 시작 · 032-511-6836\n경로\n보비동물병원\n4.6(48)\n동물병원 · 산곡동 411-5\n032-524-6959\n경로\n펫가든동물병원\n5.0(1)\n동물 병원 · 계산동 1059-1번지 2층 계양구 인천광역시 KR\n경로\n부평종합동물의료센터\n4.4(4

In [24]:
for i in range(0,10):
    driver.execute_script('test=document.querySelector("#pane > div > div.Yr7JMd-pane-content.cYB2Ge-oHo7ed > div > div > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ > div.siAUzd-neVct.section-scrollbox.cYB2Ge-oHo7ed.cYB2Ge-ti6hGc.siAUzd-neVct-Q3DXx-BvBYQ"); test.scrollTo(0, 10000000000000000000);')
    time.sleep(1)

JavascriptException: Message: javascript error: Cannot read properties of null (reading 'scrollTo')
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D8B8F3+2406643]
	Ordinal0 [0x00D1AF31+1945393]
	Ordinal0 [0x00C0C748+837448]
	Ordinal0 [0x00C0F154+848212]
	Ordinal0 [0x00C0F012+847890]
	Ordinal0 [0x00C0F98A+850314]
	Ordinal0 [0x00C650C9+1200329]
	Ordinal0 [0x00C5427C+1131132]
	Ordinal0 [0x00C64682+1197698]
	Ordinal0 [0x00C54096+1130646]
	Ordinal0 [0x00C2E636+976438]
	Ordinal0 [0x00C2F546+980294]
	GetHandleVerifier [0x00FF9612+2498066]
	GetHandleVerifier [0x00FEC920+2445600]
	GetHandleVerifier [0x00E24F2A+579370]
	GetHandleVerifier [0x00E23D36+574774]
	Ordinal0 [0x00D21C0B+1973259]
	Ordinal0 [0x00D26688+1992328]
	Ordinal0 [0x00D26775+1992565]
	Ordinal0 [0x00D2F8D1+2029777]
	BaseThreadInitThunk [0x7738FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A17A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77A17A4E+238]


In [14]:
for divtag in target.find_elements(By.XPATH, './div'):
    if divtag.text != "":
        print("--------------------------")
        atag = divtag.find_element(By.XPATH, './div/a[1]')
        subdiv = divtag.find_element(By.XPATH, "./div/div[2]")
        name = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[1]")
        star = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[3]")
        addr = subdiv.find_element(By.XPATH, "./div[2]/div[1]/div[1]/div[1]/div[1]/div[4]/div[1]")

        print(name.text)
        print(addr.text)
        print(atag.get_attribute('href').split('!')[5][2:])
        print(atag.get_attribute('href').split('!')[6].split('?')[0][2:])
        print(star.text)
# [출처] 파이썬 크롤링 셀레니움 (구글지도 맛집 정보, 오늘의집 크롤링 등)|작성자 홍인
# https://blog.naver.com/hongin90/222661902505

--------------------------


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"./div/a[1]"}
  (Session info: chrome=102.0.5005.62)
Stacktrace:
Backtrace:
	Ordinal0 [0x00D8B8F3+2406643]
	Ordinal0 [0x00D1AF31+1945393]
	Ordinal0 [0x00C0C748+837448]
	Ordinal0 [0x00C392E0+1020640]
	Ordinal0 [0x00C3957B+1021307]
	Ordinal0 [0x00C2FAE1+981729]
	Ordinal0 [0x00C542C4+1131204]
	Ordinal0 [0x00C2FA64+981604]
	Ordinal0 [0x00C54494+1131668]
	Ordinal0 [0x00C64682+1197698]
	Ordinal0 [0x00C54096+1130646]
	Ordinal0 [0x00C2E636+976438]
	Ordinal0 [0x00C2F546+980294]
	GetHandleVerifier [0x00FF9612+2498066]
	GetHandleVerifier [0x00FEC920+2445600]
	GetHandleVerifier [0x00E24F2A+579370]
	GetHandleVerifier [0x00E23D36+574774]
	Ordinal0 [0x00D21C0B+1973259]
	Ordinal0 [0x00D26688+1992328]
	Ordinal0 [0x00D26775+1992565]
	Ordinal0 [0x00D2F8D1+2029777]
	BaseThreadInitThunk [0x7738FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77A17A7E+286]
	RtlGetAppContainerNamedObjectPath [0x77A17A4E+238